In [11]:
import pandas as pd
import requests
from nltk.tokenize import word_tokenize

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re 
from string import punctuation 

import nltk
nltk.download('stopwords')
import joblib 

from wordcloud import WordCloud, STOPWORDS

for dependency in (
    "brown",
    "names",
    "wordnet",
    "averaged_perceptron_tagger",
    "universal_tagset",
):
    nltk.download(dependency)


import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\youss\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [5]:
r = requests.get('https://newsapi.org/v2/everything?q=tesla&from=2024-12-29&sortBy=publishedAt&apiKey=5ba4bef2c78d47b4b31d19dbf0c0fd50')
x = r.json()
articles = x.get("articles", [])
data = []
for article in articles:
    data.append({
        "author": article.get("author"),
        "title": article.get("title"),
        "publishedAt": article.get("publishedAt"),
        "content": article.get("content"),
        "source_name": article.get("source", {}).get("name") 
    })
df = pd.DataFrame(data)

df

,author,title,publishedAt,content,source_name
0,FAZ.NET,Aktuelle Nachrichten aus Frankfurt und Rhein-M...,2025-01-28T18:07:08Z,Mainz. Die jüngsten Äußerungen des amerikanisc...,Faz.net
1,Guille Lomener,Meta activa ‘salas de guerra’ para descifrar c...,2025-01-28T18:01:13Z,Hay un nombre que le está quitando el sueño a ...,Applesfera.com
2,William Gavin,Tesla's fourth-quarter earnings have investors...,2025-01-28T18:00:00Z,In This Story\r\nTesla (TSLA-0.68%\r\n) is set...,Quartz India
3,Bobby Green,Jay och Jason snackar V12,2025-01-28T18:00:00Z,42 idag\r\nVälkommen till Feber\r\nSkaffa Febe...,Feber.se
4,None,Mercedes-Benz se une a la demanda colectiva co...,2025-01-28T17:57:17Z,MADRID 28 Ene. (EUROPA PRESS) - \r\n Mercedes-...,Europapress.es
...,...,...,...,...,...
93,Celia López,Wall Street abre plano y Nvidia trata de recup...,2025-01-28T15:16:23Z,Wall Street ha abierto este martes prácticamen...,Elperiodico.com
94,Aisha Malik,Trump says Microsoft is in talks to acquire Ti...,2025-01-28T15:15:59Z,President Donald Trump told reporters that Mic...,TechCrunch
95,benzinga.com,Recent Filing Shows That Rep. Marjorie Taylor ...,2025-01-28T15:14:06Z,"January 27, 2025 records indicate that Represe...",Biztoc.com
96,None,US tech stocks nudge lower day after steep sel...,2025-01-28T15:12:13Z,LONDON : Tech stocks nudged lower on Tuesday w...,CNA


In [12]:
stop_words = set(stopwords.words('english'))
def clean_data(text,remove_stop_words=True,lemmatize_words=True):
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r"ur", " your ", text)
    text = re.sub(r" nd "," and ",text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r" tkts "," tickets ",text)
    text = re.sub(r" c "," can ",text)
    text = re.sub(r" e g ", " eg ", text)
    text =  re.sub(r'http\S+',' link ', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text) 
    text = re.sub(r" u "," you ",text)
    text = re.sub(r" em "," them ",text)
    text = re.sub(r" re "," are ",text)
    text = re.sub(r" s "," is ",text)
    text = re.sub(r" m "," am ",text)
    text = re.sub(r" b "," be ",text)
    text = re.sub(r" d "," and ",text)
    text = re.sub(r" u "," you ",text)
    text = re.sub(r" ur "," your ",text)
    text = re.sub(r" nd "," and ",text)
    text=text.lower()
    text=''.join([i for i in text if i not in punctuation])
    if remove_stop_words:
        text = ' '.join([word for word in text.split() if word not in stop_words])
    if lemmatize_words:
        lemmatizer = WordNetLemmatizer()
        text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])    
    return text
    

In [13]:
df["clean_content"]=df["content"].apply(lambda x: clean_data(x))
df["clean_title"]=df["title"].apply(lambda x: clean_data(x))
df["clean_author"]=df["author"].apply(lambda x: clean_data(str(x)))

In [15]:
df["clean_source_name"]=df["source_name"].apply(lambda x: clean_data(str(x)))

In [16]:
df

,author,title,publishedAt,content,source_name,clean_content,clean_title,clean_author,clean_source_name
0,FAZ.NET,Aktuelle Nachrichten aus Frankfurt und Rhein-M...,2025-01-28T18:07:08Z,Mainz. Die jüngsten Äußerungen des amerikanisc...,Faz.net,mainz die jüngsten äußerungen de amerikanische...,aktuelle nachrichten au frankf und rheinmain faz,faznet,faznet
1,Guille Lomener,Meta activa ‘salas de guerra’ para descifrar c...,2025-01-28T18:01:13Z,Hay un nombre que le está quitando el sueño a ...,Applesfera.com,hay un nombre que le está quitando el sueño lo...,meta activa ‘salas de guerra’ para descifrar c...,guille lomener,applesferacom
2,William Gavin,Tesla's fourth-quarter earnings have investors...,2025-01-28T18:00:00Z,In This Story\r\nTesla (TSLA-0.68%\r\n) is set...,Quartz India,story tesla tsla068 set report fo thquarter fu...,tesla fo thquarter earnings investor asking fa...,william gavin,quartz india
3,Bobby Green,Jay och Jason snackar V12,2025-01-28T18:00:00Z,42 idag\r\nVälkommen till Feber\r\nSkaffa Febe...,Feber.se,idag välkommen till feber skaffa feber marantz...,jay och jason snackar v12,bobby green,feberse
4,None,Mercedes-Benz se une a la demanda colectiva co...,2025-01-28T17:57:17Z,MADRID 28 Ene. (EUROPA PRESS) - \r\n Mercedes-...,Europapress.es,madrid ene europa press mercedesbenz se ha uni...,mercedesbenz se une la demanda colectiva contr...,none,e opapresses
...,...,...,...,...,...,...,...,...,...
93,Celia López,Wall Street abre plano y Nvidia trata de recup...,2025-01-28T15:16:23Z,Wall Street ha abierto este martes prácticamen...,Elperiodico.com,wall street ha abierto este martes prácticamen...,wall street abre plano nvidia trata de recuper...,celia lópez,elperiodicocom
94,Aisha Malik,Trump says Microsoft is in talks to acquire Ti...,2025-01-28T15:15:59Z,President Donald Trump told reporters that Mic...,TechCrunch,president donald trump told reporter microsoft...,trump say microsoft talk acquire tiktok,aisha malik,techcrunch
95,benzinga.com,Recent Filing Shows That Rep. Marjorie Taylor ...,2025-01-28T15:14:06Z,"January 27, 2025 records indicate that Represe...",Biztoc.com,january 27 record indicate representative marj...,recent filing show rep marjorie taylor greene ...,benzingacom,biztoccom
96,None,US tech stocks nudge lower day after steep sel...,2025-01-28T15:12:13Z,LONDON : Tech stocks nudged lower on Tuesday w...,CNA,london tech stock nudged lower tuesday chipmak...,u tech stock nudge lower day steep selloff dol...,none,cna
